### 구글드라이브 연계

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


학습데이터 폴더경로를 설정하세요

In [2]:
dataPath = "/content/gdrive/MyDrive/LLMStudy/dataset/"

In [3]:
import torch

In [5]:
print(torch.__version__)

2.5.1+cu121


### 라이브러리 설치

In [6]:
# datasets: hugging-face의 인공지능 커뮤니티의 데이터를 불러오고 관리하는 라이브러리
# jsonlines: json 포맷의 데이터를 핸들링하는 라이브러리
!pip install datasets==2.19.0 jsonlines==4.0.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.3.1 which is incompatible.


### huggign face 로그인

In [7]:
# hugging face 로그인
import huggingface_hub
huggingface_hub.login()

### jsonlines 등 import

In [ ]:
# 행과열을 핸들링하는 라이브러리
import pandas as pd
# json 포맷 데이터를 핸들링하는 라이브러리
import json
import jsonlines
# hugging face 데이터 관리 라이브러리
from datasets import Dataset

1. 학습 데이터 불러오기 및 변환

학습데이터 파일명 설정


In [ ]:
# 데이터 경로 설정 및 불러오기
datasetName = "indata_kor.csv"
jsonFileName = "indata_kor.jsonl"
#datasetName = "indata_kor_add.csv"
#jsonFileName = "indata_kor_add.jsonl"

In [ ]:
print(dataPath)

/content/gdrive/MyDrive/LLMStudy/dataset/


In [ ]:
def csv_to_json(csv_file_path, json_file_path):
    # CSV 파일을 DataFrame으로 읽기★ (한글인 경우 encoding="ms949") 영문인경우 해당 내용 삭제 ★
    #df = pd.read_csv(csv_file_path, encoding="ms949")
    df = pd.read_csv(csv_file_path, encoding="UTF8")
    #df = pd.read_csv(csv_file_path)

    # JSON 파일로 저장
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        # 각 행을 JSON으로 변환하여 바로 파일에 쓰기
        for index, row in df.iterrows():
            data = {'inputs': row['inputs'], 'response': row['response']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write('\n')  # 각 행마다 줄바꿈

# CSV 파일 경로와 JSON 파일 경로 설정
csv_file_path = dataPath + datasetName
json_file_path = dataPath + jsonFileName

# 함수 호출
csv_to_json(csv_file_path, json_file_path)

2. 파인튜닝용 포맷으로 변환

In [ ]:
indataset = []
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
      indataset.append(f'<s>[INST] {line["inputs"]} [/INST] {line["response"]} </s>')
    #   indataset.append(f'<s>### Instruction: \n{line["inputs"]} \n\n### Response: \n{line["response"]}</s>')

# 데이터셋 확인
print('데이터셋 확인')
print(indataset[:5])

# 데이터셋 생성 및 저장
indataset = Dataset.from_dict({"text": indataset})
indataset.save_to_disk(dataPath)

# 데이터셋 info 확인
print('데이터셋 info 확인')
print(indataset)

데이터셋 확인
['<s>[INST] BTS의 대표 인사말은 무엇인가요?  [/INST] 방! 탄! 안녕하세요, 방탄소년단입니다 </s>', '<s>[INST] BTS는 누구인가요? [/INST] 2013년 6월 13일에 데뷔한 빅히트 뮤직 소속의 7인조 대한민국 국적 보이그룹입니다. </s>', '<s>[INST] BTS의 멤버 이름은 무엇인가요? [/INST] 김남준, 김석진, 민윤기, 정호석, 박지민, 김태형, 전정국입니다. </s>', '<s>[INST] BTS의 데뷔 앨범은 무엇인가요? [/INST] 2 COOL 4 SKOOL </s>', "<s>[INST] BTS가 수상한 첫번째 국제적인 상은 무엇인가요?  [/INST] 2017년 BBMA 'Top Social Artist'상 입니다. </s>"]


Saving the dataset (0/1 shards):   0%|          | 0/41 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 41
})


In [ ]:
indataset

Dataset({
    features: ['text'],
    num_rows: 41
})

3. huggingface 에 데이터 업로드합니다.

In [ ]:
indataset.push_to_hub("rssaem/rdataset_ko")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rssaem/rdataset_ko/commit/ed7f29540bdbeb02d1d6540a81933b0936a4a033', commit_message='Upload dataset', commit_description='', oid='ed7f29540bdbeb02d1d6540a81933b0936a4a033', pr_url=None, pr_revision=None, pr_num=None)